### <$Q_\pi$를 추정하기 위한 비활성 MC 예측(정책평가) >
출처 : 단단한 강화학습 챕터 5 수도코드 

입력 : 임의의 목표 정책 $\pi$ 

모든 $s \in S, a \in A(s)$를 초기화 : 
- $Q(s,a) \in R$ (임의로 설정) 
- C(s,a) <- 0 

(각 에피소드에 대해) 무한 루프 : 
- b <- 정책 $\pi$가 보증된 임의의 정책 
- 정책 b를 따르는 에피소드를 생성 : $S_0, A_0, R_1, ... S_{T-1}, A_{T-1}, R_T$
- G <- 0 
- W <- 1 
- 에피소드의 각 단계에 대한 루프 ,$t= T-1, T-2, ... 0$ 

> G <- $\gamma G + R_{t-1}$

> $C(S_t, A_t) <- C(S_t, A_t) + W$ 

> $Q(S_t, A_t) <- Q(S_t, A_t) + \frac {W}{C(S_t, A_t)} [G- Q(S_t, A_T)]$

> W <- $W \frac{\pi(A_t|S_t)} {b(A_t|S_t)}$

> W = 0 이면 루프를 종료 


**구현해야 하는 것** 
- Q(s,a) : 기존 함수 유지 
- C(s,a) : dict 또는 이중 리스트로 구현 

- b <- 정책 $\pi$ 가 보증된 임의의 정책 : How? 

- W : int 

- $\pi(a,s)$ : 외부 함수로 구현해야함. 


**필요한 것** 
- 외부함수 pi , reward func 

**함수/데이터의 형태** 
- class epsilon_soft_MC : 

> def __init__(self, S, A, reward_func, pi, epsilon=0.001, gamma = 0.9, num_episode = 10, len_episode = 20) 

> def choice_sample(self, list, prob) : 기존 함수 유지 

> def make_episode(self, start_s, T, pi) : 기존 함수 유지  

> def update_returns(self) : 비활성 MC에 따라서 구현 

**전제**
- 상태 s에서 행동 a를 했을 때 결정론적으로 s' 상태로 변화한다. 

<외부 함수> 
- def reward_func(s',a,s) : 

- def pi(s,a) : 


<고민점>
- 정책 $\pi$가 보증된 임의의 정책을 어떻게 표현할 것인가? 

> 보증한다는 것은 $\pi$ 가 가지는 모든 경우의 수를 포함하고 있어야 한다는 것. 일반적인 경우로 epsilon-soft로 정의하자. 



In [3]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [4]:
from collections import defaultdict
import random
import numpy as np

In [5]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 
    
#최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 
def choose_random_max(lst) :
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 
    
def pi(s,a) : #s 상황에서 a를 선택할 확률. 전체 합은 1이여야 한다. 
    # 확률은 모두 동일하게 설정 
    
    return 1/10

In [6]:
# 입실론 소프트 활성 MC 제어 참고 

class off_policy_MC :  
    def __init__(self, S, A, pi, reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.epsilon = epsilon 
        self.reward_func = reward_func
        self.pi = pi
        self.b, self.C = self.initiate_b()
        self.gamma = 0.9 
        self.num_episode = num_episode
        self.T = len_episode
        
        self.set_episode = [] # 추후에 어떤 episode들이 있었나 확인용 
        
        self.Q, self.return_lst, self.s_prob = self.initiate() 
    
    
    def initiate_b(self) : 
        # pi를 보증하는 b 정책 초기화하기. 예측이기 때문에 b 정책이 바뀔 일은 없음.
        # C(s,a) 도 같이 초기화 
        pi_dic = defaultdict(float)
        c_dic = defaultdict(float)
        for s in self.S :
            for a in self.A :  
                pi_dic[(s,a)] = 1/len(self.A)
                c_dic[(s,a)] = 0
        return pi_dic, c_dic
    
    def initiate(self) :  
        Q = defaultdict(float)
        s_prob = defaultdict(list)
        for s in self.S : 
            s_prob[s] = [0]*len(self.A)
            for index, a in enumerate(self.A) : 
                Q[(s,a)] = 0 
                s_prob[s][index] = self.pi(s,a)

        return_lst = [[0]*len(self.A)]*len(self.S) # return_lst의 데이터 형식 변경
        return Q, return_lst, s_prob
    
    def choice_sample(self, s) : 
        b_a_list = [] 
        for _ in self.A :
            b_a_list.append(self.b[(s,_)])
            
        a= random.choices(self.A, weights = b_a_lsit[s]) # s는 index와 동일해야 한다. 
        
        # 수정. a[0] +s 가 s의 범위 안에 들어오도록 수정 
        return a[0], min(max(s+a[0],0), 99)  
    
    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a, next_s = self.choice_sample(s)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # returns 가 사라짐에 따라 대거 수정  
        for _ in range(self.num_episode) :
            start_s = random.choice(self.S)

            episode = self.make_episode(start_s, self.T)
            G = 0 
            W = 1 # W값 추가 
            
            for t in reversed(range(self.T)) : 
                G = self.gamma *G + episode["R"][t]
                s, a  = episode["S"][t], episode["A"][t]
                self.C[(s,a)] = self.C[(s,a)] + W
                self.Q[(s,a)] = self.Q[(s,a)] + W/self.C[(s,a)] * (G - self.Q[(s,a)])
                W = W*self.pi(s,a)/self.b[(s,a)] 
                if W == 0 : break        
        
                
        

In [7]:
# update_returns 구현 테스트 
print(pi(2,3))
test = off_policy_MC(S,A,reward_func, pi) 


# 오류가 왜 생기는 거지? reward_func 는 아래 식에서 관련이 없는데? 

#print(test.C)
#print(test.Q)

0.1


TypeError: reward_func() missing 1 required positional argument: 's'